In [1]:
"""
In this notebook ,
I add dropout.
When p =0.5 for dropout , nonzero outputs are doubled
1 / 1-p = 1 / 0.5 = 2

if p = 0.2
1 / 1-0.2 = 1 / 0.8 = 1.25
"""
import torch 
import numpy 
from sklearn.datasets import make_classification

In [18]:
dump_ = 1

def dump_info(tensor_,title):
    if dump_ == 1:
        print(title)
        print( list(tensor_.size() ))
        print(tensor_)
        
class Feedforward(torch.nn.Module):
        def __init__(self, input_size, hidden_size,drop_out_prob):
            print("Feedforward input_size ",input_size," hidden_size ", hidden_size," drop_out_prob",drop_out_prob)
            super(Feedforward, self).__init__()
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.relu = torch.nn.ReLU()
            self.drop = torch.nn.Dropout(p=drop_out_prob)
            self.fc2 = torch.nn.Linear(self.hidden_size, 1)
            self.sigmoid = torch.nn.Sigmoid()        
        
        def forward(self, x):
            global dump_
            
            dump_info(x,"input")
            
            hidden = self.fc1(x)            
            dump_info(hidden,"hidden fc1")
            
            #drop_output =  torch.nn.functional.dropout(hidden) 
            drop_output =  self.drop(hidden) 
            dump_info(drop_output,"drop_output")
            
            relu = self.relu(drop_output)            
            dump_info(relu,"relu")
            
            
            
            output = self.fc2(relu)            
            dump_info(output,"output fc2")
            
            sigmoid_output = self.sigmoid(output)
            dump_info(sigmoid_output,"sigmoid_output")
            #close logging
            dump_ = 2
            
            return sigmoid_output

In [3]:
# CREATE RANDOM DATA POINTS
total_samples = 5
x_train, y_train = make_classification(n_samples=total_samples,n_features=2, n_redundant=0, n_informative=1,
                             n_clusters_per_class=1)
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)

x_test, y_test = make_classification(n_samples=total_samples,n_features=2, n_redundant=0, n_informative=1,
                             n_clusters_per_class=1)
x_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(y_test)

In [4]:
model = Feedforward(2, 4,0.5)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)
print(model)
model.eval()
y_pred = model(x_test)
before_train = criterion(y_pred.squeeze(), y_test)
print('Test loss before training' , before_train.item())

#input        [5, 2]  our input shape
#hidden fc1   [5, 4]  (fc1): Linear(in_features=2, out_features=4, bias=True) 
#drop_output  [5, 4]  since =0.5 nonzero outputs are doubled
#relu         [5, 4]  relu same size
#output fc2   [5, 1]  Linear(in_features=4, out_features=1, bias=True)
#sigmoid_output  [5, 1]  Sigmoid

Feedforward input_size  2  hidden_size  4  drop_out_prob 0.5
Feedforward(
  (fc1): Linear(in_features=2, out_features=4, bias=True)
  (relu): ReLU()
  (drop): Dropout(p=0.5)
  (fc2): Linear(in_features=4, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
input
[5, 2]
tensor([[ 0.2435,  1.1330],
        [-1.1941, -0.3866],
        [ 1.5834,  0.1661],
        [-1.2566, -0.0566],
        [-1.0901, -1.0576]])
hidden fc1
[5, 4]
tensor([[-0.2873, -0.2479,  0.9685,  1.0959],
        [ 0.2281, -0.7970, -0.6134,  0.0622],
        [-0.7914, -0.1276,  1.3661,  0.8261],
        [ 0.2544, -0.7558, -0.5032,  0.2223],
        [ 0.1830, -0.8855, -0.8518, -0.2670]], grad_fn=<AddmmBackward>)
drop_output
[5, 4]
tensor([[-0.0000, -0.0000,  0.0000,  2.1918],
        [ 0.0000, -0.0000, -1.2268,  0.1245],
        [-0.0000, -0.2553,  2.7322,  1.6523],
        [ 0.5088, -1.5116, -1.0064,  0.4445],
        [ 0.0000, -0.0000, -1.7036, -0.0000]], grad_fn=<MulBackward0>)
relu
[5, 4]
tensor([[0.0000, 0.0000, 0.00

In [8]:
model.train()
epoch = 20
for epoch in range(epoch):    
    optimizer.zero_grad()    
    # Forward pass
    y_pred = model(x_train)    
    # Compute Loss
    loss = criterion(y_pred.squeeze(), y_train)
   
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))    # Backward pass
    loss.backward()
    optimizer.step()
model.eval()
y_pred = model(x_test)
after_train = criterion(y_pred.squeeze(), y_test) 
print('Test loss after Training' , after_train.item())    

Epoch 0: train loss: 0.7601617574691772
Epoch 1: train loss: 0.8074312210083008
Epoch 2: train loss: 0.7312209010124207
Epoch 3: train loss: 0.7304747104644775
Epoch 4: train loss: 0.6818045377731323
Epoch 5: train loss: 0.8593791127204895
Epoch 6: train loss: 0.8514102101325989
Epoch 7: train loss: 0.774050235748291
Epoch 8: train loss: 0.7576993703842163
Epoch 9: train loss: 0.7600385546684265
Epoch 10: train loss: 0.6953762769699097
Epoch 11: train loss: 0.719214141368866
Epoch 12: train loss: 0.7432788014411926
Epoch 13: train loss: 0.696179211139679
Epoch 14: train loss: 0.7173563241958618
Epoch 15: train loss: 0.730347752571106
Epoch 16: train loss: 0.6704256534576416
Epoch 17: train loss: 0.7253202199935913
Epoch 18: train loss: 0.7618869543075562
Epoch 19: train loss: 0.8282432556152344
Test loss after Training 0.9036760330200195


In [21]:
dump_ = 1
model = Feedforward(2, 4,0.2)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)
print(model)
model.train()
epoch = 20
for epoch in range(epoch):    
    optimizer.zero_grad()    
    # Forward pass
    y_pred = model(x_train)    
    # Compute Loss
    loss = criterion(y_pred.squeeze(), y_train)
   
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))    # Backward pass
    loss.backward()
    optimizer.step()

#input        [5, 2]  our input shape
#hidden fc1   [5, 4]  (fc1): Linear(in_features=2, out_features=4, bias=True) 
#drop_output  [5, 4]  since =0.1 nonzero outputs are multiplied by 1.25
#relu         [5, 4]  relu same size
#output fc2   [5, 1]  Linear(in_features=4, out_features=1, bias=True)
#sigmoid_output  [5, 1]  Sigmoid
#*** check the dropout values, they are either zero or scaled by 1.25

Feedforward input_size  2  hidden_size  4  drop_out_prob 0.2
Feedforward(
  (fc1): Linear(in_features=2, out_features=4, bias=True)
  (relu): ReLU()
  (drop): Dropout(p=0.2)
  (fc2): Linear(in_features=4, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
input
[5, 2]
tensor([[ 0.3473, -2.9985],
        [-0.1728,  1.0055],
        [-0.1656,  0.9867],
        [ 1.2761, -0.1070],
        [ 0.5659, -0.5252]])
hidden fc1
[5, 4]
tensor([[ 0.9088,  0.2307, -0.3162,  0.1287],
        [-0.0486, -0.8633,  0.4945, -0.9935],
        [-0.0453, -0.8562,  0.4880, -0.9866],
        [-0.1144, -0.0165, -0.4570, -0.2470],
        [ 0.1801, -0.2204, -0.1160, -0.3846]], grad_fn=<AddmmBackward>)
drop_output
[5, 4]
tensor([[ 0.0000,  0.0000, -0.3952,  0.1609],
        [-0.0607, -1.0792,  0.6181, -0.0000],
        [-0.0000, -1.0703,  0.6100, -1.2333],
        [-0.1430, -0.0000, -0.5712, -0.0000],
        [ 0.0000, -0.0000, -0.1450, -0.4807]], grad_fn=<MulBackward0>)
relu
[5, 4]
tensor([[0.0000, 0.0000, 0.00

In [23]:
dump_ = 1
model.eval()
y_pred = model(x_test)
before_train = criterion(y_pred.squeeze(), y_test)
print('Test loss before training' , before_train.item())
#since we are at eval mode, dropout does not have any effect

input
[5, 2]
tensor([[ 0.2435,  1.1330],
        [-1.1941, -0.3866],
        [ 1.5834,  0.1661],
        [-1.2566, -0.0566],
        [-1.0901, -1.0576]])
hidden fc1
[5, 4]
tensor([[-0.1966, -0.7121,  0.3068, -0.8723],
        [ 0.6004, -0.9878,  0.8927, -1.0135],
        [-0.2531,  0.0488, -0.5807, -0.2151],
        [ 0.5210, -1.0835,  0.9749, -1.1070],
        [ 0.7676, -0.8027,  0.7384, -0.8310]], grad_fn=<AddmmBackward>)
drop_output
[5, 4]
tensor([[-0.1966, -0.7121,  0.3068, -0.8723],
        [ 0.6004, -0.9878,  0.8927, -1.0135],
        [-0.2531,  0.0488, -0.5807, -0.2151],
        [ 0.5210, -1.0835,  0.9749, -1.1070],
        [ 0.7676, -0.8027,  0.7384, -0.8310]], grad_fn=<AddmmBackward>)
relu
[5, 4]
tensor([[0.0000, 0.0000, 0.3068, 0.0000],
        [0.6004, 0.0000, 0.8927, 0.0000],
        [0.0000, 0.0488, 0.0000, 0.0000],
        [0.5210, 0.0000, 0.9749, 0.0000],
        [0.7676, 0.0000, 0.7384, 0.0000]], grad_fn=<ThresholdBackward0>)
output fc2
[5, 1]
tensor([[0.3666],
        